# 3. Evaluación de Modelos

Ahora que nos hemos familiarizado con nuestros datos, el próximo paso lógico es explorar el espacio de los algoritmos que eventualmente producirán un buen modelo para la tarea que buscamos resolver.

Nuestra meta en este notebook no es desarrollar una solución vanguardista, sino, más bien, revisar diversas arquitecturas con el fin de ver cuáles serán promovidas a la siguiente etapa del proceso, centrada en la optimización.

Sin más preámbulos, pongámonos manos a la obra.

## Transfer Learning

En computer vision siempre es buena idea empezar apoyándonos en el conocimiento de modelos pre-entrenados. Esta técnica se conoce como _transfer learning_.

Keras ya viene con una serie de modelos entrenados en ImageNet, lo cual es genial. Esta vez utilizaremos la inferfaz de alto nivel de Keras que se halla dentro de TensorFlow, en vez de su versión independiente.

## Revisión

Para evaluar un amplio espectro de posibles algoritmos, necesitamos primero implementar algunos métodos. Empecemos por darle forma a la data que usaremos.

### Datos

Dado que estamos revisando redes neuronales profundas, debemos preservar tanta memoria como sea posible. Es por este motivo que generaremos lotes de datos bajo demanda, directamente desde el disco, utilizando `flow_from_directory`.

Esta función espera que las imágenes correspondientes a una clase se encuentren dentro de un subdirectorio con el nombre de la misma. Es por eso que debemos reajustar nuestra estructura de directorios. 

También apartaremos el 10% de nuestro conjunto de datos para validar que los modelos estén aprendiendo.

In [ ]:
import glob
import cv2
import os
from sklearn.utils import shuffle

destination_directory = './dataset'

def load_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    return image

VALIDATION_PROPORTION = 0.1
vehicles_images_path = shuffle(glob.glob('data/vehicles/*/*.png'))
split_point = int(len(vehicles_images_path) * VALIDATION_PROPORTION)

for i, image_path in enumerate(vehicles_images_path):
    image = load_image(image_path)
    
    if i < split_point:
        destination_path = os.path.join(destination_directory, 'valid', 'vehicle', f'{i}.png')
    else:
        destination_path = os.path.join(destination_directory, 'train', 'vehicle', f'{i}.png')
    
    cv2.imwrite(destination_path, image)
    
non_vehicles_images_path = shuffle(glob.glob('data/non-vehicles/*/*.png'))
split_point = int(len(non_vehicles_images_path) * VALIDATION_PROPORTION)
for i, image_path in enumerate(non_vehicles_images_path):
    image = load_image(image_path)
    
    if i < split_point:
        destination_path = os.path.join(destination_directory, 'valid', 'non_vehicle', f'{i}.png')
    else:
        destination_path = os.path.join(destination_directory, 'train', 'non_vehicle', f'{i}.png')
    
    cv2.imwrite(destination_path, image)

Como ya mencionamos anteriormente, el aspecto positivo del método `flow_from_directory` es que crea mapeos para las etiquetas con base en la estructura de subdirectorios donde yacen los datos.

### Modelos

Ahora, procederemos a definir una función para obtener los modelos que deseamos evaluar.

La función `get_models` retornará un `dict` de `dict`s, donde las claves del diccionario externo son los nombres del modelo pre-entrenado que estamos usando, y los valores son diccionarios que contienen la función de preprocesamiento asociada al modelo pre-entrenado, una función para construir el modelo propiamente y las dimensiones de entrada que éste espera.

La sub-función `get_model_with_new_top` toma una _base_ (modelo pre-entrenado) y le coloca una red neuronal totalmente conectada (también conocidas como perceptrones multicapa) encima. También congela todas las capas en la base. Por último, compila el modelo para que utilice el optimizador `adam`.

In [1]:
from tensorflow.keras import applications
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
import gc

SEED = 314159

def get_models(models=None):
    # Takes a base, pretrained model, and attaches a new FCN on top of it.
    def get_model_with_new_top(base_model):
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(512, activation='relu')(x)
        x = Dense(256, activation='relu')(x)
        predictions = Dense(1, activation='sigmoid')(x)
        
        model = Model(inputs=base_model.input, outputs=predictions)
        
        for layer in base_model.layers:
            layer.trainable = False
            
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
        return model
        
    if models is None:
        models = dict()
        
    models['mobilenet'] = {
        'preprocessing_function': applications.mobilenet.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
        'input_shape': (224, 224)
    }
    
    models['resnet50'] = {
        'preprocessing_function': applications.resnet50.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
        'input_shape': (224, 224)
    }
    
    models['inceptionV3'] = {
        'preprocessing_function': applications.inception_v3.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
        'input_shape': (224, 224)
    }
    
    models['xception'] = {
        'preprocessing_function': applications.xception.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))), 
        'input_shape': (224, 224)
    }
    
    models['nasnet_large'] = {
        'preprocessing_function': applications.nasnet.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.NASNetLarge(weights='imagenet', include_top=False, input_shape=(331, 331, 3))),
        'input_shape': (331, 331)
    }
    
    models['nasnet_mobile'] = {
        'preprocessing_function': applications.nasnet.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.NASNetMobile(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
        'input_shape': (224, 224)
    }
    
    models['densenet121'] = {
        'preprocessing_function': applications.densenet.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
        'input_shape': (224, 224)
    }
    
    models['densenet169'] = {
        'preprocessing_function': applications.densenet.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.DenseNet169(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
        'input_shape': (224, 224)
    }
    
    models['densenet201'] = {
        'preprocessing_function': applications.densenet.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))),
        'input_shape': (224, 224)
    }
    
    models['inception_resnet_v2'] = {
        'preprocessing_function': applications.inception_resnet_v2.preprocess_input,
        'model_constructor': lambda: get_model_with_new_top(applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(299, 299, 3))),
        'input_shape': (299, 299)
    }
    
    models['vgg16'] = {
        'preprocessing_function': applications.vgg16.preprocess_input, 
        'model_constructor': lambda: get_model_with_new_top(applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))), 
        'input_shape': (224, 224)
    }
    
    models['vgg19'] = {
        'preprocessing_function': applications.vgg19.preprocess_input, 
        'model_constructor': lambda: get_model_with_new_top(applications.VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))), 
        'input_shape': (224, 224)
    }
    
    return models

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Como podemos notar en la celda anterior, estamos utilizando un amplio rango de _bases_, desde modelos pequeños hasta unos bastante grandes.

Puesto que nuestros datos son lo suficientemente pequeños y guardan cierta relación con ImageNet, sólo entrenaremos las capas del perceptrón multicapa que se encuentra encima del modelo pre-entrenado. De manera más concreta, el mayor beneficio que estamos cosechando al usar _transfer learning_ es la extracción de _features_.

### Evaluación

Es hora de evaluar el desempeño de cada candidato. Para ello, crearemos una función llamada `train_and_evaluate_models`, la cual recibe un `dict` de modelos (tal y como los retorna `get_models`), un número de _epochs_, y entrena cada clasificador por ese número de iteraciones.

Para medir el rendimiento, usaremos la data de validación que apartamos hace un par de celdas. Esto nos permitirá conocer qué tan bien le va a cada modelo sobre imágenes nunca vistas.

También estamos iteresados en mantener nuestra solución lo más magra posible, por lo que conocer el número de parametros es relevante.

Finalmente, es destacable el hecho de que estamos limpiando la sesión y desechando cada modelo una vez hemos terminado con él. Esto es fundamental dado que, de lo contrario, nuestra computadora se rompería por falta de memoria.

In [ ]:
def train_and_evaluate_models(models, epochs=5):
    for model_name, model_data in models.items():
        m = model_data['model_constructor']()
        train_data_generator = ImageDataGenerator(preprocessing_function=model_data['preprocessing_function']).flow_from_directory('./dataset/train', 
                                                                                                                                   target_size=model_data['input_shape'],
                                                                                                                                   batch_size=32,
                                                                                                                                   class_mode='binary')
        
        valid_data_generator = ImageDataGenerator(preprocessing_function=model_data['preprocessing_function']).flow_from_directory('./dataset/valid', 
                                                                                                                                   target_size=model_data['input_shape'],
                                                                                                                                   batch_size=32,
                                                                                                                                   class_mode='binary')
        step_size_train = train_data_generator.n // train_data_generator.batch_size

        print(f'Training {model_name}')
        history = m.fit_generator(generator=train_data_generator,
                                  steps_per_epoch=step_size_train,
                                  validation_data=valid_data_generator,
                                  validation_steps=(valid_data_generator.n // valid_data_generator.batch_size),
                                  epochs=epochs)

        print('Number of parameters: {:,}'.format(m.count_params()))
        print('---------------\n\n')

        del m
        del history
        K.clear_session()
        gc.collect()

¡Estamos listos! Hora de poner a prueba a los candidatos.

In [3]:
models = get_models()
train_and_evaluate_models(models)

Instructions for updating:
Colocations handled automatically by placer.
Found 6593 images belonging to 2 classes.
Found 732 images belonging to 2 classes.
Training mobilenet
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
207/207 [==============================] - 27s 132ms/step - loss: 0.1239 - acc: 0.9540 - val_loss: 0.1482 - val_acc: 0.9467
Epoch 2/5
207/207 [==============================] - 25s 120ms/step - loss: 0.0554 - acc: 0.9791 - val_loss: 0.0569 - val_acc: 0.9822
Epoch 3/5
207/207 [==============================] - 24s 118ms/step - loss: 0.0510 - acc: 0.9800 - val_loss: 0.1439 - val_acc: 0.9399
Epoch 4/5
207/207 [==============================] - 24s 117ms/step - loss: 0.0396 - acc: 0.9848 - val_loss: 0.0527 - val_acc: 0.9809
Epoch 5/5
207/207 [==============================] - 24s 117ms/step - loss: 0.0564 - acc: 0.9801 - val_loss: 0.0580 - val_acc: 0.9822
Number of parameters: 3,885,249
---------------




/usr/lib/python3/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Found 6593 images belonging to 2 classes.
Found 732 images belonging to 2 classes.
Training resnet50
Epoch 1/5
207/207 [==============================] - 74s 356ms/step - loss: 0.0916 - acc: 0.9630 - val_loss: 0.0534 - val_acc: 0.9809
Epoch 2/5
207/207 [==============================] - 70s 338ms/step - loss: 0.0523 - acc: 0.9809 - val_loss: 0.0938 - val_acc: 0.9672
Epoch 3/5
207/207 [==============================] - 70s 337ms/step - loss: 0.0279 - acc: 0.9909 - val_loss: 0.0544 - val_acc: 0.9768
Epoch 4/5
207/207 [==============================] - 71s 342ms/step - loss: 0.0716 - acc: 0.9833 - val_loss: 0.2844 - val_acc: 0.9139
Epoch 5/5
207/207 [==============================] - 70s 336ms/step - loss: 0.0627 - acc: 0.9810 - val_loss: 0.0463 - val_acc: 0.9822
Number of parameters: 24,768,385
---------------


Found 6593 images belonging to 2 classes.
Found 732 images belonging to 2 classes.
Training inceptionV3
Epoch 1/5
207/207 [==============================] - 54s 262ms/step - loss

### Conclusión

Hemos entrenado a cada candidato durante 5 _epochs_ con el fin de evaluar el poder de aprendizaje de cada modelo. Como establecimos con anterioridad, queremos mantener aquellos modelos que manifiesten un buen desempeño y que, a su vez, minimicen la complejidad (es decir, el número de parámetros).

Es menester destacar que los modelos más grandes (tales como InceptionResnetV2, NASNetLarge y Xception), después de 5 _epochs_, presentan peor rendimiento que modelos cuya arquitectura es más simple, como MobileNet o NASNetMobile. Esto es entendible, puesto que mientras más grande es un modelo, más tiempo le toma aprender debido al alto volumen de parámetros.

Debido a que nuestra data es relativamente pequeña y no tan compleja, pareciera que los modelos más pequeños lo hacen mejor.

El podio de candidatos que promoveremos a la fase de optimización son:

 - **VGG16**. Número de parámetros: 15,108,929. Exactitud en el conjunto de validación: 99.32%.
 - **MobileNet**. Número de parámetros: 3,885,249.Exactitud en el conjunto de validación: 98.22%.
 - **NASNetMobile**. Número de parámetros: 4,942,485. Exactitud en el conjunto de validación: 97.27%.
 
Aunque hay algunos modelos que presentan una exactitud un poco más alta que **MobileNet** y **NASNetMobile**, son mucho más grandes, por lo que el incremento en complejidad no vale la pena, dada la pequeña mejora.